### Multi-Target Predictions | Feature Selection Via ANOVA or t-tests

#### RE TUNING AND REGULARIZATION APPLIED 

#### Import necessary libraries

In [ ]:
import joblib
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import f_oneway
from sklearn.feature_selection import f_regression
from statsmodels.stats.outliers_influence import variance_inflation_factor

import mlflow
import mlflow.sklearn
from scipy.stats import boxcox
from sklearn.preprocessing import PowerTransformer
from catboost import CatBoostRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

# Import Utility Functions
from model_utils import classify_skewness, transform_features_for_skewness, transform_targets
from model_utils import range_without_outliers, remove_outliers
from model_utils import SkewnessTransformer
from model_utils import categorical_value_counts_to_df, group_low_frequency_categories
from model_utils import get_overfitting_status

import warnings
warnings.filterwarnings("ignore")

#### Columns To Drop

In [ ]:
COLS_TO_DROP = ["longitude", "latitude"]

#### Hyperparameters and Other definitions

In [ ]:
MODEL_ESTIMATOR_ALPHA: list = [0.0001, 0.001, 0.01, 0.1, 1, 10]
MODEL_ESTIMATOR_DEPTH: list[int] = [4, 6, 8]
MODEL_ESTIMATOR_NUM_LEAVES: list[int] = [31, 50]
MODEL_ESTIMATOR_MAX_DEPTH: list[int] = [5, 10, 15]
MODEL_ESTIMATOR_N_ESTIMATORS_RF: list[int] = [100, 200]
MODEL_ESTIMATOR_N_ESTIMATORS_ADA: list[int] = [50, 100]
MODEL_ESTIMATOR_N_ESTIMATORS_BAGGING: list[int] = [10, 50]
MODEL_ESTIMATOR_N_NEIGHBORS: list[int] = [5, 10]
MODEL_ESTIMATOR_SVR_LINEAR_KERNEL: list[str] = ['linear'] # Restrict kernel to 'linear'
MODEL_ESTIMATOR_C_SVR: list = [0.1, 1, 10]
MODEL_ESTIMATOR_C_SVR_ADJ: list = [0.01, 0.1, 1]
MODEL_CATBOOST_LEARNING_RATE: list = [0.0001, 0.001, 0.01, 0.1]
MODEL_CATBOOST_ESTIMATOR_L2_LEAF_REG: list[int] = [1, 3, 5, 10] # Add L2 regularization
MODEL_CATBOOST_ESTIMATOR_L2_LEAF_REG_ADJ: list[int] = [3, 5, 7, 10]
MODEL_DECISIONTREE_MAX_FEATURES: list = ['sqrt', 'log2', None]  # Feature selection
MIN_SAMPLE_LEAF: list[int] = [1, 3, 5, 7]

NUM_SIMPLE_IMPUTER: str = "mean"
CAT_SIMPLE_IMPUTER: str = "most_frequent"
ONE_HOT_ENCODER_HANDLE_UNKNOWN: str = "ignore"
POWER_TRANSFORMER_METHOD: str = "yeo-johnson"
CV: int = 10
# GRIDSEARCHCV_SCORING: str = "neg_mean_absolute_error"
GRIDSEARCHCV_SCORING: str = "neg_root_mean_squared_error"
RANDOM_STATE: int = 0
TRAIN_SIZE: float = 0.7
TEST_SIZE: float = 0.2
VAL_SIZE: float = 0.1
N_REPEATS: int = 10

MODEL_SAVE_PATH: str = f"./checkpoints/trained_multiple_models/"
BEST_MODEL_SUMMARY_CSV_PATH: str = f"./"
DATASET_VERSION: str = "v3"
SAVE_SUMMARY_TO_CSV: str = "True"

# Make dir if it doesn't exist
Path(MODEL_SAVE_PATH).mkdir(parents=True, exist_ok=True)
# Path(BEST_MODEL_SUMMARY_CSV_PATH).mkdir(parents=True, exist_ok=True)

# # MLflow experiment setup
# URI = "http://127.0.0.1:5000"
# mlflow.set_experiment(f"Maz | IPage")
# mlflow.set_tracking_uri(URI)

#### Load Dataset

In [ ]:
file_path = f"../../../data/merged_{DATASET_VERSION}.csv"
# Create a Path object
data_file_path = Path(file_path)
data = pd.read_csv(data_file_path)

df = data.copy()

In [ ]:
# print(f"Dataset: {df.head()}")
df.head()

In [ ]:
length_df = len(df)
print(f"Length of Dataset: {length_df}")

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
print(df.isnull().values.any())
print(df.isnull().sum().sum())
print("\n")
print(df.isnull().sum())

In [ ]:
# Check for duplicates
duplicates = df.duplicated()
print(duplicates.sum())

#### Drop Columns 

In [ ]:
df.drop(COLS_TO_DROP, axis=1, inplace=True)
df.describe()

#### Drop raw longitude and latitude if present

In [ ]:
# if 'longitude' in features and 'latitude' in features:
#     features.remove(COLS_TO_DROP[0])
#     features.remove(COLS_TO_DROP[1])

#### Define target variable and features

In [ ]:
targets = ["Boron", "Zinc", "SOC"]
features = [col for col in df.columns if col not in targets]

In [ ]:
print(f"Targe: {targets}")
print(f"Features: {features}")

#### Identify categorical and numerical features

In [ ]:
categorical_features = df[features].select_dtypes(include=['object', 'category']).columns.tolist()
numerical_features = df[features].select_dtypes(include=['number']).columns.tolist()

In [ ]:
print(f"Categorical Features:\n {categorical_features}\n")
print(f"Numerical Features:\n {numerical_features}\n")

#### Targets

In [ ]:
# Set up the number of rows and columns
num_targets = len(targets)
cols = 3  # Number of columns per row
rows = (num_targets + cols - 1) // cols  # Calculate required rows

# Create subplots
fig, axes = plt.subplots(rows, cols, figsize=(10, 4))

# Flatten axes for easier iteration
axes = axes.flatten()

# Loop through numerical features and plot
for i, target in enumerate(targets):
    sns.histplot(df[target], kde=True, bins=20, ax=axes[i])
    axes[i].set_title(f"Distribution of {target}")

# Remove unused subplots
for i in range(len(targets), len(axes)):
    fig.delaxes(axes[i])

# Adjust Layout
plt.tight_layout()
plt.show()

##### 1. Check for Skewnes in Targets

In [ ]:
# Check skewness and provide recommendations
for target in targets:
    skewness = df[target].skew()
    skewness_category, recommendation = classify_skewness(skewness)
    print(f"Skewness of '{target}': {skewness:.4f}")
    print(f"  Skewness Category: {skewness_category}")
    print(f"  Recommendation: {recommendation}")
    print("-" * 100)

#### 1. Analyze Ccategorical_features

In [ ]:
print(f"Unique Values in 'Area' col:\n {df['Area'].unique()}\n")
print("*" * 120)
print(f"Unique Values in 'Soil group' col:\n {df['Soil group'].unique()}\n")
print("*" * 120)
print(f"Unique Values in 'Land class' col\n: {df['Land class'].unique()}\n")
print("*" * 120)
print(f"Unique Values in 'Soil type' col\n: {df['Soil type'].unique()}\n")

In [ ]:
# Get the value counts in Categorical Columns of DataFrame
counts_df = categorical_value_counts_to_df(df)
# Print the resulting DataFrame
print(f"Categorical Value Counts")
print("*" * 24, "\n")
# print(counts_df)
counts_df

In [ ]:
# Group low frequency categories
df = group_low_frequency_categories(df, threshold=5)
# Print the resulting DataFrame
# print(f"Grouped Low Frequency Categories")
# print("*" * 32, "\n")
# print(modified_df)
# modified_df.head(3)

# # Check Again | Get the value counts DataFrame
# counts_df = categorical_value_counts_to_df(df)
# # Print the resulting DataFrame
# print(f"Categorical Value Counts")
# print("*" * 24, "\n")
# # print(counts_df)
# counts_df

In [ ]:
# Set up the figure and axes
fig, axes = plt.subplots(2, 2, figsize=(14, 10))  # 2 rows and 2 columns

# Flatten the axes for easy indexing
axes = axes.flatten()

# Loop through the categorical features and plot
for i, feature in enumerate(categorical_features):
    sns.countplot(data=df, x=feature, ax=axes[i])
    axes[i].set_title(f"Distribution of {feature}")
    axes[i].tick_params(axis='x', rotation=45)

# Adjust layout
plt.tight_layout()
plt.show()

#### 2. Analyze Numerical Features

##### 1. Histograms: To visualize the distributions of numerical variables:

In [ ]:
# Set up the number of rows and columns
num_features = len(numerical_features)
cols = 3  # Number of columns per row
rows = (num_features + cols - 1) // cols  # Calculate required rows

# Create subplots
fig, axes = plt.subplots(rows, cols, figsize=(18, 12))

# Flatten axes for easier iteration
axes = axes.flatten()

# Loop through numerical features and plot
for i, feature in enumerate(numerical_features):
    sns.histplot(df[feature], kde=True, bins=20, ax=axes[i])
    axes[i].set_title(f"Distribution of {feature}")

# Remove unused subplots
for i in range(len(numerical_features), len(axes)):
    fig.delaxes(axes[i])

# Adjust Layout
plt.tight_layout()
plt.show()

##### 2. Check for Outliers

In [ ]:
# Set the number of columns per row
cols = 3  # 3 plots in each row
num_features = len(numerical_features)
rows = (num_features + cols - 1) // cols  # Calculate the required number of rows dynamically

# Create subplots dynamically
fig, axes = plt.subplots(rows, cols, figsize=(18, 4 * rows))  # Adjust height for rows

# Flatten the axes for easier iteration
axes = axes.flatten()

# Loop through the numerical features and plot boxplots
for i, feature in enumerate(numerical_features):
    sns.boxplot(x=df[feature], ax=axes[i])
    axes[i].set_title(f"Boxplot of {feature}", pad=15)  # Add padding for the title

# Remove unused subplots (if any)
for i in range(len(numerical_features), len(axes)):
    fig.delaxes(axes[i])

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

##### 3. Display Range of Values in Numerical Features without Outliers

In [ ]:
#df.drop(["longitude", "latitude"], axis=1).describe()

In [ ]:
# Display the range for each numerical feature
print(f"\nRange of Values in Each Column (Numerical) without Outliers")
print("*" * 59, "\n")
for feature in numerical_features:
    min_val, max_val = range_without_outliers(df, feature)
    print(f"{feature}: Min: {min_val}, Max: {max_val}\n")

##### 4. Remove Outliers

In [ ]:
# Dynamically apply the function to each numerical feature
for feature in numerical_features:
    df = remove_outliers(df, feature)

length_df_remove_outliers = len(df)
print(f"Length of Dataset After Removal of Ouliers")
print("*"* 42)
print(length_df_remove_outliers)

# Print the cleaned dataset
print("\nDataset after removing outliers:")
print("*"* 32, "\n")
df.describe().T

##### 5. Check for Skewness in Numerical Features

In [ ]:
# df[numerical_features].head(3)

In [ ]:
# Function to classify skewness
def classify_skewness(skewness):
    if abs(skewness) > 2:
        return "Highly Skewed", "Recommend Log Transformation (if positive) or Yeo-Johnson"
    elif abs(skewness) > 1:
        return "Moderately Skewed", "Recommend Yeo-Johnson Transformation"
    elif abs(skewness) > 0.5:
        return "Slightly Skewed", "Transformation optional"
    else:
        return "Symmetrical", "No transformation needed"

# Check skewness and provide recommendations
for feature in numerical_features:
    skewness = df[feature].skew()
    skewness_category, recommendation = classify_skewness(skewness)
    print(f"Skewness of '{feature}': {skewness:.4f}")
    print(f"  Skewness Category: {skewness_category}")
    print(f"  Recommendation: {recommendation}")
    print("-" * 100)

#### 3. Analyze Relationships Between Features and Targets

##### 1. Categorical Features vs Targets

In [ ]:
# Create subplots dynamically
for i, target in enumerate(targets):
    # One row, multiple columns
    fig, axes = plt.subplots(1, len(categorical_features), figsize=(20, 5))  
    
    for j, feature in enumerate(categorical_features):
        sns.boxplot(data=df, x=feature, y=target, ax=axes[j])
        axes[j].set_title(f"{target} vs {feature}", pad=10)
        axes[j].tick_params(axis='x', rotation=45)
    
    # Adjust layout for the current row
    plt.suptitle(f"Target: {target}", fontsize=16)
    # Add space for the suptitle
    plt.tight_layout(rect=[0, 0, 1, 0.95])  
    plt.show()


In [ ]:
# Step 1: Check unique categories for each feature
for feature in categorical_features:
    print(f"{feature}: {df[feature].nunique()} unique categories")

# Step 2: Filter categorical features with more than one unique category
valid_categorical_features = [feature for feature in categorical_features if df[feature].nunique() > 1]
print(f"Valid Category Features: {valid_categorical_features}")

# Step 3: Perform ANOVA for each target
anova_results_all_targets = {}

for target in targets:
    anova_results = {}  # To store results for the current target
    # print(f"\nPerforming ANOVA for target: {target}")
    
    for feature in valid_categorical_features:
        # Group target values by the categorical feature
        groups = [df[df[feature] == category][target].dropna() for category in df[feature].unique()]
        
        # Perform one-way ANOVA
        if len(groups) > 1:  # Ensure at least two groups exist
            f_stat, p_value = f_oneway(*groups)
            anova_results[feature] = {'F-Statistic': f_stat, 'p-value': p_value}
    
    # Convert results to a DataFrame
    anova_results_df = pd.DataFrame(anova_results).T
    anova_results_df['Significant'] = anova_results_df['p-value'] < 0.05
    
    # Store results for the current target
    anova_results_all_targets[target] = anova_results_df

# Step 4: Display all results after the loop
for target, results_df in anova_results_all_targets.items():
    print(f"\nANOVA Results for {target}:")
    print("*" * 25)
    print(results_df)

    # Optionally save to a CSV or Excel file
    # results_df.to_csv(f"anova_results_{target}.csv", index=True)
    # results_df.to_excel(f"anova_results_{target}.xlsx", index=True)

In [ ]:
# Filter features with more than one unique category
categorical_features = [feature for feature in categorical_features if df[feature].nunique() > 1]
print(f"Updated Categorical Features: {categorical_features}")

##### 2. Numerical Features vs Targets

a. Statistical Tests (ANOVA or t-tests)

In [ ]:
# Perform statistical test for each target
statistical_results = {}
relevant_features = {}  # Dictionary to store relevant features for each target

for target in targets:
    # Separate features and target
    X = df[numerical_features]
    y = df[target]
    
    # Perform F-statistic test
    f_stat, p_values = f_regression(X, y)
    
    # Store results in a DataFrame
    result_df = pd.DataFrame({
        'Feature': numerical_features,
        'F-Statistic': f_stat,
        'p-value': p_values
    }).sort_values(by='p-value')
    
    # Save results for the current target
    statistical_results[target] = result_df
    
    # Filter features with p-value < 0.05
    relevant_features[target] = result_df[result_df['p-value'] < 0.05]['Feature'].tolist()

# Display statistical analysis for each target
for target, result_df in statistical_results.items():
    print(f"\nStatistical Analysis for Target: {target}")
    print("*" * 39)
    print(result_df)

# Dynamically Split Numerical Features
numFeatures_GroupA_Boron = relevant_features.get("Boron", [])
numFeatures_GroupA_Zinc = relevant_features.get("Zinc", [])
numFeatures_GroupA_SOC = relevant_features.get("SOC", [])

# Display relevant features for each target
print("\nRelevant Features for Each Target (p-value < 0.05):")
print("*" * 51)

if numFeatures_GroupA_Boron:
    print(f"Boron: {numFeatures_GroupA_Boron}")
else:
    print("Boron: No relevant features found (p-value >= 0.05)")

if numFeatures_GroupA_Zinc:
    print(f"Zinc: {numFeatures_GroupA_Zinc}")
else:
    print("Zinc: No relevant features found (p-value >= 0.05)")

if numFeatures_GroupA_SOC:
    print(f"SOC: {numFeatures_GroupA_SOC}")
else:
    print("SOC: No relevant features found (p-value >= 0.05)")

##### b. Handle Multicollinearity for Regression Models

In [ ]:
# Flag for regression model
REGRESSION_MODEL = True

# Function to calculate VIF and handle multicollinearity
def handle_multicollinearity_vif(features, df, vif_threshold=15.0):
    X = df[features]
    vif_data = pd.DataFrame()
    vif_data["Feature"] = features
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    # Identify features with VIF above the threshold
    high_vif_features = vif_data[vif_data["VIF"] > vif_threshold]["Feature"].tolist()
    
    # Dynamically create a second group without multicollinear features
    group_without_multicollinearity = [f for f in features if f not in high_vif_features]

    return group_without_multicollinearity, vif_data

# Check and dynamically create groups for each target
# Boron
if numFeatures_GroupA_Boron:
    print("\nHandling Multicollinearity for Boron:")
    print("*" * 38)
    numFeatures_GroupB_Boron, vif_boron = handle_multicollinearity_vif(numFeatures_GroupA_Boron, df)
    print(f"Group A (original): {numFeatures_GroupA_Boron}")
    print(f"Group B (adjusted): {numFeatures_GroupB_Boron}")
    if len(numFeatures_GroupB_Boron) < len(numFeatures_GroupA_Boron):  # Multicollinearity exists
        print("\nVIF Data for Boron:")
        print(vif_boron)
        if REGRESSION_MODEL:
            print("\nUse Group B for regression.")
        else:
            print("\nGroup B created but not needed for tree-based models.")
    else:
        print("\nNo multicollinearity found.")
        if REGRESSION_MODEL:
            print("\nUse Group A for regression.")
        else:
            print("\nUse Group A for tree-based models.")

# Zinc
if numFeatures_GroupA_Zinc:
    print("\nHandling Multicollinearity for Zinc:")
    print("*" * 36)
    numFeatures_GroupB_Zinc, vif_zinc = handle_multicollinearity_vif(numFeatures_GroupA_Zinc, df)
    print(f"Group A (original): {numFeatures_GroupA_Zinc}")
    print(f"Group B (adjusted): {numFeatures_GroupB_Zinc}")
    if len(numFeatures_GroupB_Zinc) < len(numFeatures_GroupA_Zinc):  # Multicollinearity exists
        print("\nVIF Data for Zinc:")
        print(vif_zinc)
        if REGRESSION_MODEL:
            print("\nUse Group B for regression.")
        else:
            print("\nGroup B created but not needed for tree-based models.")
    else:
        print("\nNo multicollinearity found.")
        if REGRESSION_MODEL:
            print("\nUse Group A for regression.")
        else:
            print("\nUse Group A for tree-based models.")

# SOC
if numFeatures_GroupA_SOC:
    print("\nHandling Multicollinearity for SOC:")
    print("*" * 35)
    numFeatures_GroupB_SOC, vif_soc = handle_multicollinearity_vif(numFeatures_GroupA_SOC, df)
    print(f"Group A (original): {numFeatures_GroupA_SOC}")
    print(f"Group B (adjusted): {numFeatures_GroupB_SOC}")
    if len(numFeatures_GroupB_SOC) < len(numFeatures_GroupA_SOC):  # Multicollinearity exists
        print("\nVIF Data for SOC:")
        print(vif_soc)
        if REGRESSION_MODEL:
            print("\nUse Group B for regression.")
        else:
            print("\nGroup B created but not needed for tree-based models.")
    else:
        print("\nNo multicollinearity found.")
        if REGRESSION_MODEL:
            print("\nUse Group A for regression.")
        else:
            print("\nUse Group A for tree-based models.")

#### Define Models

In [ ]:
models = {
    'LinearRegression': {
        'model': MultiOutputRegressor(LinearRegression()),
        'type': 'regression',
        'params': {}  # No significant hyperparameters for Linear Regression
    },
    'Lasso': {
        'model': MultiOutputRegressor(Lasso()),
        'type': 'regression',
        'params': {"model__estimator__alpha": MODEL_ESTIMATOR_ALPHA}
    },
    'DecisionTree': {
        'model': MultiOutputRegressor(DecisionTreeRegressor()),
        'type': 'tree',
        'params': {
            'model__estimator__max_depth': MODEL_ESTIMATOR_DEPTH,
            'model__estimator__min_samples_leaf': MIN_SAMPLE_LEAF,
            'model__estimator__max_features': MODEL_DECISIONTREE_MAX_FEATURES,  # Feature selection
        }
    },
    'RandomForest': {
        'model': MultiOutputRegressor(RandomForestRegressor(random_state=RANDOM_STATE)),
        'type': 'tree',
        'params': {
            'model__estimator__n_estimators': MODEL_ESTIMATOR_N_ESTIMATORS_RF,
            'model__estimator__max_depth': MODEL_ESTIMATOR_MAX_DEPTH,
            'model__estimator__min_samples_leaf': MIN_SAMPLE_LEAF,  # Regularization
        }
    },
    'AdaBoost': {
        'model': MultiOutputRegressor(AdaBoostRegressor(random_state=RANDOM_STATE)),
        'type': 'tree',
        'params': {
            'model__estimator__n_estimators': MODEL_ESTIMATOR_N_ESTIMATORS_ADA,
        }
    },
    'Bagging': {
        'model': MultiOutputRegressor(BaggingRegressor(random_state=RANDOM_STATE)),
        'type': 'tree',
        'params': {
            'model__estimator__n_estimators': MODEL_ESTIMATOR_N_ESTIMATORS_BAGGING,
        }
    },
    'KNeighbors': {
        'model': MultiOutputRegressor(KNeighborsRegressor()),
        'type': 'tree',
        'params': {
            'model__estimator__n_neighbors': MODEL_ESTIMATOR_N_NEIGHBORS,
        }
    },
    'SVR': {
        'model': MultiOutputRegressor(SVR()),
        'type': 'regression',
        'params': {
        'model__estimator__kernel': MODEL_ESTIMATOR_SVR_LINEAR_KERNEL,  # Restrict kernel to 'linear'
        'model__estimator__C': MODEL_ESTIMATOR_C_SVR,  # Include the value 0.1 and others for tuning
    }
    },
    'XGB': {
        'model': MultiOutputRegressor(XGBRegressor(random_state=RANDOM_STATE)),
        'type': 'tree',
        'params': {
            'model__estimator__n_estimators': MODEL_ESTIMATOR_N_ESTIMATORS_RF,
            'model__estimator__max_depth': MODEL_ESTIMATOR_MAX_DEPTH,
            'model__estimator__learning_rate': MODEL_ESTIMATOR_ALPHA,
        }
    },
    'CatBoost': {
    'model': MultiOutputRegressor(CatBoostRegressor(verbose=0, random_state=RANDOM_STATE)),
    'type': 'tree',
    'params': {
        'model__estimator__depth': MODEL_ESTIMATOR_DEPTH,  # Example: [4, 6, 8]
        'model__estimator__learning_rate': MODEL_CATBOOST_LEARNING_RATE,  # Example: [0.01, 0.1]
        'model__estimator__l2_leaf_reg': MODEL_CATBOOST_ESTIMATOR_L2_LEAF_REG,  # Regularization
        }
    },
}

In [ ]:
# for model_name, model_info in models.items():
#     print(f"{model_name}: {model_info['params']}")

#### Define Feature Sets For Numerical Variables

In [ ]:
# Feature sets for each group

feature_sets = {
    'GroupA': {
        'Boron': numFeatures_GroupA_Boron + categorical_features,  # Group A for Boron
        'Zinc': numFeatures_GroupA_Zinc + categorical_features,    # Group A for Zinc
        'SOC': numFeatures_GroupA_SOC + categorical_features       # Group A for SOC
    },
    'GroupB': {
        'Boron': numFeatures_GroupB_Boron + categorical_features,  # Group B for Boron
        'Zinc': numFeatures_GroupB_Zinc + categorical_features,    # Group B for Zinc
        'SOC': numFeatures_GroupB_SOC + categorical_features       # Group B for SOC
    }
}

#### Dataset split

In [ ]:
print(f"Categorical Features: {categorical_features}")
print(f"Numerical Features: {numerical_features}")

In [ ]:
# X = df[numerical_features + categorical_features]  # Full feature set
y = df[targets]  # Target variables

In [ ]:
print(f"X Columns: {X.columns}")
print(f"y Columns: {y.columns}")

#### Apply Transformations To Numerical Features

In [ ]:
# transformed_num_features, num_transformation_report = transform_features_for_skewness(numerical_features, df)
# num_transformation_report

In [ ]:
# transformed_num_features.head(3)

In [ ]:
# # Update Numerical Features in DataFrameabs
# df[numerical_features] = transformed_num_features

In [ ]:
# # Again Check skewness and provide recommendations
# for num_feature in numerical_features:
#     skewness = df[num_feature].skew()
#     skewness_category, recommendation = classify_skewness(skewness)
#     print(f"Skewness of '{num_feature}': {skewness:.4f}")
#     print(f"  Skewness Category: {skewness_category}")
#     print(f"  Recommendation: {recommendation}")
#     print("-" * 100)

#### Apply Transformations To Targets 

In [ ]:
# specific_transformations = {"Boron": "boxcox"}
transformed_targets, target_transformation_report = transform_targets(y, skewness_threshold=0.75, specific_transformations=None)

In [ ]:
target_transformation_report

In [ ]:
transformed_targets.head(3)

In [ ]:
### Update Targets
df[targets] = transformed_targets
y = df[targets]
y.head(3)

In [ ]:
# Again Check skewness and provide recommendations
for target in targets:
    skewness = df[target].skew()
    skewness_category, recommendation = classify_skewness(skewness)
    print(f"Skewness of '{target}': {skewness:.4f}")
    print(f"  Skewness Category: {skewness_category}")
    print(f"  Recommendation: {recommendation}")
    print("-" * 100)

#### Split data into train, Val and test sets

In [ ]:
X = df[numerical_features + categorical_features]  # Full feature set

In [ ]:
# First split: Train + Temp (Validation + Test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=(1 - TRAIN_SIZE), random_state=RANDOM_STATE)

# Second split: Validation + Test from Temp
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=(TEST_SIZE / (VAL_SIZE + TEST_SIZE)), random_state=RANDOM_STATE)

print(f"Training Set: {X_train.shape}, Validation Set: {X_val.shape}, Test Set: {X_test.shape}")

In [ ]:
# Define preprocessing steps
numerical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy=NUM_SIMPLE_IMPUTER)),
        # ("skewness", SkewnessTransformer()),  # Apply skewness transformations
        ("scaler", StandardScaler()),  # Scale features after transformation,
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy=CAT_SIMPLE_IMPUTER)),
        ("onehot", OneHotEncoder(handle_unknown=ONE_HOT_ENCODER_HANDLE_UNKNOWN)),
    ]
)

# preprocessor = ColumnTransformer(
#     transformers=[
#         ("num", numerical_transformer, numerical_features),
#         ("cat", categorical_transformer, categorical_features),
#     ]
# )

# pipeline.named_steps['preprocessor'].transformers_[0][1].named_steps['skewness'].skewness_report


In [ ]:
print(f"Categorical Features: {categorical_features}")
print(f"Numerical Features: {numerical_features}")

#### Train Models with MLFlow

In [ ]:
# DAGSHUB
import dagshub
# dagshub.init(repo_owner='mm-mazhar', repo_name='IPAGE', mlflow=True)
dagshub.init(repo_owner="Omdena", repo_name="IPage", mlflow=True)

In [ ]:
# MLflow experiment setup
# URI_LOCAL = "http://127.0.0.1:5000"
# URI_MAZ = "https://dagshub.com/mm-mazhar/IPAGE.mlflow"
URI_OMDENA = "https://dagshub.com/Omdena/IPage.mlflow"
mlflow.set_experiment("Maz | Mult-Target | OverFitting Checks and Re-Tuning")
mlflow.set_tracking_uri(URI_OMDENA)

# Dictionaries to track the best model and scores for each target
best_models = {}
best_val_r2_scores = {}  # Track the best validation R² scores
best_r2_scores = {}
best_model_paths = {}
best_features = {}
overfitting_metrics = {}

counter = 1

# Iterate over models and feature sets
for model_name, model_info in models.items():
    base_model = model_info['model']
    model_type = model_info['type']
    hyperparameters = model_info.get('params', {})
    
    # Select feature set based on model type
    feature_set_key = 'GroupB' if model_type == 'regression' else 'GroupA'
    feature_set = feature_sets[feature_set_key]
    
    print(f"\nTraining {model_name} (Type: {model_type}) using {feature_set_key}... | {counter}")
    print("*" * 65)
    
    for target, features in feature_set.items():
        if target not in best_r2_scores:
            best_val_r2_scores[target] = -np.inf
            best_r2_scores[target] = -np.inf
            best_models[target] = None
            best_model_paths[target] = None
            best_features[target] = None
            overfitting_metrics[target] = "Unknown"

        selected_numerical_features = [f for f in features if f in numerical_features]
        selected_categorical_features = [f for f in features if f in categorical_features]

        preprocessor = ColumnTransformer(
            transformers=[
                ("num", numerical_transformer, selected_numerical_features),
                ("cat", categorical_transformer, selected_categorical_features),
            ]
        )

        pipeline = Pipeline(steps=[
            ("preprocessor", preprocessor),
            ("model", base_model)
        ])

        X_train_features = X_train[features]
        X_val_features = X_val[features]
        X_test_features = X_test[features]

        y_train_target_values = y_train[[target]]
        y_val_target_values = y_val[[target]]
        y_test_target_values = y_test[[target]]

        grid_search = GridSearchCV(
            estimator=pipeline,
            param_grid=hyperparameters,
            scoring=GRIDSEARCHCV_SCORING,
            cv=CV
        )
    
        with mlflow.start_run(run_name=f"Maz | {model_name} | {target} | {feature_set_key}"):
            grid_search.fit(X_train_features, y_train_target_values)
            best_pipeline = grid_search.best_estimator_

            val_predictions = best_pipeline.predict(X_val_features)
            val_r2 = r2_score(y_val_target_values, val_predictions)

            mlflow.log_metric("val_r2", val_r2)

            test_predictions = best_pipeline.predict(X_test_features)
            mae = mean_absolute_error(y_test_target_values, test_predictions)
            mse = mean_squared_error(y_test_target_values, test_predictions)
            r2 = r2_score(y_test_target_values, test_predictions)

            mlflow.log_metric("mae", mae)
            mlflow.log_metric("mse", mse)
            mlflow.log_metric("r2", r2)

            print(f"Validation R² Score: {val_r2}")
            print(f"Test R² Score: {r2}")
            print(f"MAE: {mae}")
            print(f"MSE: {mse}")

            # Call the function to get overfitting status
            overfitting_status, overfitting_numeric = get_overfitting_status(val_r2, r2)
            
            # Log numeric overfitting status as a metric
            mlflow.log_metric("overfitting_status_numeric", overfitting_numeric)
            # Log string overfitting status as a parameter
            # mlflow.log_param("overfitting_status", overfitting_status)
            # Record in the dictionary
            overfitting_metrics[target] = overfitting_status

            # If high overfitting, re-tune hyperparameters
            if overfitting_status == "High Overfitting":
                print(f"    Model {model_name} for target {target} is overfitting. Re-tuning hyperparameters to reduce overfitting...")
                print("    ", "*" * 112)
                
                # Adjust hyperparameters dynamically
                if model_name in ["Lasso", "ElasticNet"]:
                    hyperparameters["model__estimator__alpha"] = [alpha * 10 for alpha in MODEL_ESTIMATOR_ALPHA]

                elif model_name in ["DecisionTree", "RandomForest"]:
                    hyperparameters["model__estimator__max_depth"] = [
                        max_depth - 2 for max_depth in MODEL_ESTIMATOR_MAX_DEPTH if max_depth > 3
                    ]
                    hyperparameters["model__estimator__min_samples_leaf"] = [
                        leaf + 2 for leaf in MIN_SAMPLE_LEAF
                    ]
                elif model_name == "CatBoost":
                    hyperparameters["model__estimator__l2_leaf_reg"] = MODEL_CATBOOST_ESTIMATOR_L2_LEAF_REG[1:]
                        
                elif model_name == "SVR":
                    hyperparameters["model__estimator__C"] = MODEL_ESTIMATOR_C_SVR_ADJ

                # Re-run GridSearchCV with updated hyperparameters
                print(f"    Re-running GridSearchCV for {model_name} on target {target}...")
                print("    ", "*" * 67)
                grid_search = GridSearchCV(
                    estimator=pipeline,
                    param_grid=hyperparameters,
                    scoring=GRIDSEARCHCV_SCORING,
                    cv=CV
                )
                grid_search.fit(X_train_features, y_train_target_values)

                # Update best pipeline and metrics after re-tuning
                best_pipeline = grid_search.best_estimator_
                val_predictions = best_pipeline.predict(X_val_features)
                val_r2 = r2_score(y_val_target_values, val_predictions)
                
                test_predictions = best_pipeline.predict(X_test_features)
                mae = mean_absolute_error(y_test_target_values, test_predictions)
                mse = mean_squared_error(y_test_target_values, test_predictions)
                r2 = r2_score(y_test_target_values, test_predictions)

                print(f"    Updated Validation R²: {val_r2}, Updated Test R²: {r2}")
                print("    ", "*" * 81)
                mlflow.log_metric("val_r2", val_r2)
                mlflow.log_metric("r2", r2)
                mlflow.log_metric("mae", mae)
                mlflow.log_metric("mse", mse)
                
                print(f"    Updated Validation R² Score: {val_r2}")
                print(f"    Updated Test R² Score: {r2}")
                print(f"    Updated MAE: {mae}")
                print(f"    Updated MSE: {mse}")

                # Call the function to get overfitting status
                overfitting_status, overfitting_numeric = get_overfitting_status(val_r2, r2)
                # Log numeric overfitting status as a metric
                mlflow.log_metric("overfitting_status_numeric", overfitting_numeric)
                # Log string overfitting status as a parameter
                # mlflow.log_param("overfitting_status", overfitting_status)
                # Record in the dictionary
                overfitting_metrics[target] = overfitting_status
                
            # Log parameters, metrics, and model
            logged_params = {
                "model_name": model_name,
                "target": target,
                "feature_set": feature_set_key,
                "features": ", ".join(map(str, features)),
                # "Validation R²": val_r2,
                # "Test R²": r2,
                "Overfitting Status": overfitting_status,
                **grid_search.best_params_,
            }
            
            # Log parameters
            for param_name, param_value in logged_params.items():
                mlflow.log_param(param_name, param_value)
        
            # Print logged parameters
            print("-" * 50)
            print(f"Parameters for {model_name} and Target {target}:")
            print("-" * 50)
            for param_name, param_value in logged_params.items():
                print(f"    {param_name}: {param_value}")

            # print("-" * 100)

            if val_r2 > best_val_r2_scores[target]:
                best_val_r2_scores[target] = val_r2

            if r2 > best_r2_scores[target]:
                best_r2_scores[target] = r2
                best_models[target] = best_pipeline
                best_features[target] = features
                best_model_name = f"Maz_{model_name}_{target}_{feature_set_key}"
                best_model_path = Path(
                    f"{MODEL_SAVE_PATH}{best_model_name}_r2_{round(r2, 4)}_dataset_{DATASET_VERSION}.pkl"
                )
                best_model_paths[target] = best_model_path
                joblib.dump(best_pipeline, best_model_path)

                input_example = pd.DataFrame([X_test.iloc[0].values], columns=X_test.columns)
                mlflow.sklearn.log_model(
                    best_pipeline,
                    artifact_path=f"Maz_{model_name}_{target}_{feature_set_key}",
                    input_example=input_example
                )
                print(f"New Best Model for {target}: {best_model_name} with Test R²: {r2:.4f}")
                print(f"Best model for {target} saved to {best_model_path}")

    counter += 1

#### Summary of best models

In [ ]:
print("\nExperimentation Complete!")
print("Summary of Best Models:")
print("*" * 80)

summary_data = {
    "Target": [],
    "Best Model": [],
    "Validation R² Score": [],
    "Test R² Score": [],
    "Overfitting Status": [],
    "Features": [],
    "Model Path": []
}

for target in best_models:
    summary_data["Target"].append(target)
    summary_data["Best Model"].append(best_models[target].steps[-1][1].__class__.__name__)
    summary_data["Validation R² Score"].append(best_val_r2_scores[target])
    summary_data["Test R² Score"].append(best_r2_scores[target])
    summary_data["Overfitting Status"].append(overfitting_metrics[target])
    summary_data["Features"].append(", ".join(best_features[target]))
    summary_data["Model Path"].append(str(best_model_paths[target]) if best_model_paths[target] else "Not Saved")

summary_df = pd.DataFrame(summary_data)

if SAVE_SUMMARY_TO_CSV:
    summary_csv_path = Path(
        f"{BEST_MODEL_SUMMARY_CSV_PATH}trained_multiple_models_summary_dataset_{DATASET_VERSION}_reg.csv"
    )
    summary_df.to_csv(summary_csv_path, index=False)
    print(f"\nSummary saved to {summary_csv_path}")

summary_df